## Web scraping with selenium

In [1]:
from datetime import date, datetime, timedelta
import time
import io
import os
from io import BytesIO
# from PIL import Image
import requests
from bs4 import BeautifulSoup
import sys
import logging

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

log = logging.getLogger(__name__)

## Run service

In [2]:
options = webdriver.ChromeOptions()
options.gpu = False
options.headless = True
options.headless = False
desired = options.to_capabilities()

# donwload chromedriver matched your chrome
service = Service('./chromedriver.exe')
service.start()


## chrome should be shown as headless is set at False
driver = webdriver.Remote(
    service.service_url, desired_capabilities=desired)

In [197]:
#URL = "https://www.theguardian.com/australia-news/2022/sep/19/why-many-australian-workers-are-expected-to-call-in-sick-this-friday"

URL = "https://www.theguardian.com/environment/2022/sep/18/they-will-get-you-in-a-headlock-australians-warned-off-pet-kangaroos-after-second-death-in-100-years"


## Open page in browser

In [198]:
driver.get(URL)

In [199]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [6]:
soup.prettify()

'<html lang="en" style="top: 0px;">\n <head>\n  <!--\n\nWe are hiring, ever thought about joining us?\nhttps://workforus.theguardian.com/careers/product-engineering/\n\n\n                                    GGGGGGGGG\n                           GGGGGGGGGGGGGGGGGGGGGGGGGG\n                       GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG\n                    GGGGGGGGGGGGGGGGG      GG   GGGGGGGGGGGGG\n                  GGGGGGGGGGGG        GGGGGGGGG      GGGGGGGGGG\n                GGGGGGGGGGG         GGGGGGGGGGGGG       GGGGGGGGG\n              GGGGGGGGGG          GGGGGGGGGGGGGGGGG     GGGGGGGGGGG\n             GGGGGGGGG           GGGGGGGGGGGGGGGGGGG    GGGGGGGGGGGG\n            GGGGGGGGG           GGGGGGGGGGGGGGGGGGGGGG  GGGGGGGGGGGGG\n           GGGGGGGGG            GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG\n           GGGGGGGG             GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG\n          GGGGGGGG              GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG\n          GGGGGGGG              GGGGGGGGGGGGG

## retrieve data from page

In [200]:
article = soup.article

### title

In [201]:
title = article.find("div", {"data-gu-name" : "headline"}).h1.text
print(title)

‘They will get you in a headlock’: Australians warned off pet kangaroos after second death in 100 years


### main body

In [202]:
all_paragraphs = article.find_all("p", {"class" : "dcr-1vtk2mf"})
main_body = ""
#combine all paragraphs into single text
for paragrah in all_paragraphs:
    main_body = main_body + " " + paragrah.text

all_paragraphs = article.find_all("p", {"class" : "dcr-1acr2wr"})
#combine all paragraphs into single text
for paragrah in all_paragraphs:
    main_body = main_body + " " + paragrah.text

all_paragraphs = article.find_all("p", {"class" : "dcr-ofo30t"})
#combine all paragraphs into single text
for paragrah in all_paragraphs:
    main_body = main_body + " " + paragrah.text
#remove trailing and tailing space
main_body = main_body.strip()
print(main_body)

Kangaroos are often considered friend, not foe. But the marsupial’s reputation took a hit last week when a 77-year-old Western Australian man was killed by the pet western grey he hand-reared from a joey. As Peter Eades lay dying on his Redmond farm, 398km south of Perth, police were forced to shoot the three-year-old male kangaroo, which was preventing an ambulance crew from reaching the injured man. WA police spokesperson Ryan Langley says family members discovered a seriously injured Eades at about 5pm last Sunday. “It is believed the man had been attacked by the kangaroo earlier in the day.” It is only the second death in 100 years caused by a kangaroo in Australia. The last was in 1936, when a 38-year-old New South Wales man, William Cruickshank, reportedly died from head injuries after trying to save his dogs from a kangaroo. A University of Melbourne behavioural ecologist, Graeme Coulson, says this week’s attack was unexpected but unsurprising. Pet kangaroos are driven by the sa

### images

#### main image

In [10]:
#media = article.find("div", {"data-gu-name" : "media"})
#fig = media.figure
#print(fig.img['src'])
#print(fig.figcaption.get_text())

#### all image

In [203]:
i = 0;
for img in article.find_all("img", {"class":"dcr-4zleql"}):
    print(img['src'])
    fname = str(i)+".jpg"
    with open(fname,"wb") as f:
        f.write(requests.get(img['src']).content)
    i = i + 1

https://i.guim.co.uk/img/media/3c1c75033e148dca5c432382ea525819d2077589/0_40_2200_1320/master/2200.jpg?width=465&quality=85&dpr=1&s=none
https://i.guim.co.uk/img/media/377b2aed306276cd6791f3a0536cc2a6df718d96/928_191_1051_838/master/1051.jpg?width=445&quality=85&dpr=1&s=none


In [55]:
import torch
import torchvision
from PIL import Image
from transformers import VisionEncoderDecoderModel, ViTFeatureExtractor, AutoTokenizer, PegasusForConditionalGeneration, PegasusTokenizer

In [13]:
#core of image captioning
img_model = VisionEncoderDecoderModel.from_pretrained("F:/UTS study/32933 Research Project/hugging face/vit-gpt2-image-captioning")
img_feature_extractor = ViTFeatureExtractor.from_pretrained("F:/UTS study/32933 Research Project/hugging face/vit-gpt2-image-captioning")
img_tokenizer = AutoTokenizer.from_pretrained("F:/UTS study/32933 Research Project/hugging face/vit-gpt2-image-captioning")
img_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
img_model.to(img_device)

max_length = 16
num_beams = 4
gen_kwargs = {"max_length": max_length, "num_beams": num_beams}
def predict_step(image_paths):
  images = []
  for image_path in image_paths:
    i_image = Image.open(image_path)
    if i_image.mode != "RGB":
      i_image = i_image.convert(mode="RGB")

    images.append(i_image)

  pixel_values = img_feature_extractor(images=images, return_tensors="pt").pixel_values
  pixel_values = pixel_values.to(img_device)

  output_ids = img_model.generate(pixel_values, **gen_kwargs)

  preds = img_tokenizer.batch_decode(output_ids, skip_special_tokens=True)
  preds = [pred.strip() for pred in preds]
  return preds

In [204]:
src_text = [
    main_body
]
print(src_text)

['Kangaroos are often considered friend, not foe. But the marsupial’s reputation took a hit last week when a 77-year-old Western Australian man was killed by the pet western grey he hand-reared from a joey. As Peter Eades lay dying on his Redmond farm, 398km south of Perth, police were forced to shoot the three-year-old male kangaroo, which was preventing an ambulance crew from reaching the injured man. WA police spokesperson Ryan Langley says family members discovered a seriously injured Eades at about 5pm last Sunday. “It is believed the man had been attacked by the kangaroo earlier in the day.” It is only the second death in 100 years caused by a kangaroo in Australia. The last was in 1936, when a 38-year-old New South Wales man, William Cruickshank, reportedly died from head injuries after trying to save his dogs from a kangaroo. A University of Melbourne behavioural ecologist, Graeme Coulson, says this week’s attack was unexpected but unsurprising. Pet kangaroos are driven by the 

In [15]:
#core of text summarization
sum_device = "cuda" if torch.cuda.is_available() else "cpu"
sum_tokenizer = PegasusTokenizer.from_pretrained("F:/UTS study/32933 Research Project/hugging face/pegasus-xsum")
sum_model = PegasusForConditionalGeneration.from_pretrained("F:/UTS study/32933 Research Project/hugging face/pegasus-xsum").to(sum_device)

C:\Users\Marco\anaconda3\envs\huggingface\lib\site-packages\transformers\generation_utils.py:1202: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 64 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [205]:
batch = sum_tokenizer(src_text, truncation=True, padding="longest", return_tensors="pt").to(sum_device)
translated = sum_model.generate(**batch)
tgt_text = sum_tokenizer.batch_decode(translated, skip_special_tokens=True)

### summerized results from main body

In [206]:
tgt_text

['The death of a 77-year-old man by a pet kangaroo in Western Australia has raised questions about the animal’s behaviour.']

In [207]:
images_path = []
for k in range(i):
    images_path.append(str(k)+".jpg")
images_path

['0.jpg', '1.jpg']

In [208]:
tgt_text = tgt_text + predict_step(images_path)

### summerized results from main body and images

In [209]:
tgt_text

['The death of a 77-year-old man by a pet kangaroo in Western Australia has raised questions about the animal’s behaviour.',
 'a pair of zebras running in a grassy field',
 'a man standing in front of a herd of sheep']

## Combined summarization

In [210]:
src_text = ""
for txt in tgt_text:
    src_text = src_text + " " +txt
src_text = src_text.strip()
print(src_text)

The death of a 77-year-old man by a pet kangaroo in Western Australia has raised questions about the animal’s behaviour. a pair of zebras running in a grassy field a man standing in front of a herd of sheep


In [211]:
batch = sum_tokenizer(src_text, truncation=True, padding="longest", return_tensors="pt").to(sum_device)
translated = sum_model.generate(**batch)
final_text = sum_tokenizer.batch_decode(translated, skip_special_tokens=True)

In [212]:
final_text

['All images are copyrighted.']